In [15]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from scipy.stats import rankdata

warnings.filterwarnings('ignore')

# ----------------------------------------------------
# 1. 설정 및 데이터 로드
# ----------------------------------------------------
RANDOM_STATE = 42
N_SPLITS = 10 

train_df = pd.read_csv(r"C:\Users\abc01\OneDrive\바탕 화면\train.csv")
test_df  = pd.read_csv(r"C:\Users\abc01\OneDrive\바탕 화면\test.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape:  {test_df.shape}")

# ----------------------------------------------------
# 2. 전처리 함수 정의 (누수 방지 & 최적화 버전)
# ----------------------------------------------------

def prepare_raw_simple(df):
    df = df.copy()
    if "birth_date" in df.columns:
        df["birth_date"] = pd.to_datetime(df["birth_date"])
        df["age"] = 2025 - df["birth_date"].dt.year
    if "gender" in df.columns:
        df["gender_int"] = df["gender"].map({"M": 0, "F": 1})
    return df

def add_eda_rule_features_v2(df):
    df = df.copy()
    # chem_01_range
    df['chem_01_range_str'] = pd.cut(
        df['chem_01'], bins=[-np.inf, 2.0, 4.0, np.inf], 
        labels=['Normal_Low', 'Warning_Zone', 'Danger_High']
    ).astype(str)
    chem_map = {'Normal_Low': 0, 'Warning_Zone': 1, 'Danger_High': 2}
    df['chem_01_range'] = df['chem_01_range_str'].map(chem_map).astype(int)

    # trace_metal_range
    df['trace_metal_range_str'] = pd.cut(
        df['trace_metal'], bins=[-np.inf, 90, 130, np.inf],
        labels=['Safe', 'Caution', 'Risk']
    ).astype(str)
    metal_map = {'Safe': 0, 'Caution': 1, 'Risk': 2}
    df['trace_metal_range'] = df['trace_metal_range_str'].map(metal_map).astype(int)

    # risk_segment
    def check_ambiguous_risk(row):
        if (row['swelling'] == 'S') and (2.0 <= row['chem_01'] < 5.0):
            return 'Target_1_Suspect'
        elif (row['swelling'] == 'Y') or (row['chem_01'] >= 5.0):
            return 'High_Risk'
        else:
            return 'Normal'
    df['risk_segment_str'] = df.apply(check_ambiguous_risk, axis=1).astype(str)
    risk_map = {'Normal': 0, 'Target_1_Suspect': 1, 'High_Risk': 2}
    df['risk_segment'] = df['risk_segment_str'].map(risk_map).astype(int)
    return df

def make_boosting_features_v2(df):
    df = df.copy()
    high_is_healthy = ["protein_level", "blood_cells"]
    low_is_healthy  = ["enzyme_A", "enzyme_B", "lipid_index", "trace_metal", "clot_time"]
    def minmax(s):
        s = s.astype(float)
        mn, mx = s.min(), s.max()
        if mx == mn: return pd.Series(0.5, index=s.index)
        return (s - mn) / (mx - mn + 1e-6)

    if all(col in df.columns for col in high_is_healthy + low_is_healthy):
        df["health_index"] = (sum(minmax(df[col]) for col in high_is_healthy) - sum(minmax(df[col]) for col in low_is_healthy))
        df["health_per_stage"] = df["health_index"] / (df["disease_stage"] + 1e-6)

    df["chem01_trace_combo"] = df["chem_01"] * df["trace_metal"]
    df["chem01_chem02_combo"] = df["chem_01"] * df["chem_02"]
    df["chem01_enzymeB_combo"] = df["chem_01"] * df["enzyme_B"]
    df["enzyme_ratio"] = df["enzyme_A"] / (df["enzyme_B"].replace(0, np.nan) + 1e-6)
    df["lipid_blood_ratio"] = df["lipid_index"] / (df["blood_cells"].replace(0, np.nan) + 1e-6)
    df["chem02_trace_ratio"] = df["chem_02"] / (df["trace_metal"].replace(0, np.nan) + 1e-6)
    df["sum_chem"] = df["chem_01"] + df["chem_02"]
    df["diff_chem"] = df["chem_01"] - df["chem_02"]
    df["sum_enzyme"] = df["enzyme_A"] + df["enzyme_B"]
    df["diff_enzyme"] = df["enzyme_A"] - df["enzyme_B"]

    if "age" in df.columns:
        df["obs_per_age"] = df["obs_days"] / (df["age"] + 1e-6)
        df["behavior_per_age"] = df["behavior_index"] / (df["age"] + 1e-6)
    df["disease_velocity"] = df["disease_stage"] / (df["obs_days"] + 1e-6)

    symptom_cols = ["fluid_accum", "organ_enlarge", "vascular_marks"]
    temp_symptoms = df[symptom_cols].replace({"Y": 1, "N": 0}).infer_objects(copy=False)
    df["symptom_count"] = temp_symptoms.sum(axis=1)
    if "swelling" in df.columns:
        df["swelling_ord"] = df["swelling"].map({"N": 0, "S": 1, "Y": 2})
    else:
        df["swelling_ord"] = 0
    df["symptom_severity"] = df["symptom_count"] + df["swelling_ord"]
    return df

def add_discrete_caution_features(X):
    X = X.copy()
    X["stage_mid"] = X["disease_stage"].between(2, 3).astype(int)
    X["beh_mid"] = (X["behavior_index"] == 1).astype(int)
    X["caution_symptom_zone"] = (X["symptom_count"].between(1, 2) & (X["swelling_ord"] >= 1)).astype(int)
    X["caution_stage_beh"] = ((X["stage_mid"] == 1) & (X["beh_mid"] == 1)).astype(int)
    if X['organ_enlarge'].dtype == 'object':
        organ_flag = (X['organ_enlarge'] == 'Y')
    else:
        organ_flag = (X['organ_enlarge'] == 1)
    X["caution_organ_only"] = (organ_flag & (X["disease_stage"] == 1) & (X["behavior_index"] <= 1)).astype(int)
    return X

def add_post_eda_features(df):
    df = df.copy()
    df['protein_treatment_interaction'] = df['protein_level'] * (1 + df['treatment'])
    df['blood_treatment_interaction']   = df['blood_cells'] * (1 + df['treatment'])
    df['chem01_treatment_interaction']  = df['chem_01'] * (1 + df['treatment'])
    df['trace_treatment_interaction']   = df['trace_metal'] * (1 + df['treatment'])
    df['coagulation_index'] = df['protein_level'] / (df['clot_time'] + 1e-6)
    df['coagulation_product'] = df['protein_level'] * df['clot_time']
    if df['organ_enlarge'].dtype == 'object':
        organ_val = (df['organ_enlarge'] == 'Y').astype(int)
    else:
        organ_val = df['organ_enlarge']
    df['deadly_combo'] = df['swelling_ord'] * organ_val * 2
    df['log_obs_days'] = np.log1p(df['obs_days'])
    df['obs_days_squared'] = df['obs_days'] ** 2
    return df

# %% [Section 2-2] (추가) ML_Health 기반 고성능 파생 피처 이식

def add_ml_health_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # 1. [핵심] 행동 지수 대비 면역력 (중요도 Top 4)
    # behavior_index가 0일 수 있으므로 1을 더해 나눔
    df['immune_per_behavior'] = df['immune_index'] / (df['behavior_index'] + 1.0)
    
    # 2. 혈구 수 대비 응고 시간 (세포 밀도와 응고 관계)
    df['clot_per_cell'] = df['clot_time'] / (df['blood_cells'] + 1e-6)
    
    # 3. 단백질 대비 지질 비율 (영양 불균형)
    df['lipid_per_protein'] = df['lipid_index'] / (df['protein_level'] + 1e-6)
    
    # 4. 지질과 화학수치 차이
    df['lipid_delta'] = df['lipid_index'] - df['chem_01']
    
    # 5. 관찰 기간 대비 미네랄 밀도
    df['trace_density'] = df['trace_metal'] / (df['obs_days'] + 1.0)
    
    # 6. 행동 지수와 치료의 상호작용
    df['behavior_treatment'] = df['behavior_index'] * (1 + df['treatment'])

    return df

def add_missing_health_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    FFFFFFINAL_ENS.ipynb에 없는 ML_Health 기반 핵심 피처 추가
    """
    df = df.copy()
    
    # 1. [핵심] 행동 지수 대비 면역력 (ML_Health 중요도 Top 4)
    # behavior_index가 낮을수록(0), 면역력이 질병 방어에 더 중요할 수 있음
    df['immune_per_behavior'] = df['immune_index'] / (df['behavior_index'] + 1.0)
    
    # 2. 혈구 수 대비 응고 시간
    # 혈액 내 세포 밀도와 응고 능력의 관계
    df['clot_per_cell'] = df['clot_time'] / (df['blood_cells'] + 1e-6)
    
    # 3. 단백질 대비 지질 비율 (영양 불균형 지표)
    df['lipid_per_protein'] = df['lipid_index'] / (df['protein_level'] + 1e-6)
    
    # 4. 지질과 화학수치(chem_01) 차이
    # 서로 다른 생체 지표 간의 괴리 정도
    df['lipid_delta'] = df['lipid_index'] - df['chem_01']
    
    # 5. 관찰 기간 대비 미네랄(Trace Metal) 밀도
    # 관찰 기간 동안 미네랄 수치가 얼마나 농축되었는지 (시간 보정)
    df['trace_density'] = df['trace_metal'] / (df['obs_days'] + 1.0)

    # 6. 행동 지수와 치료의 상호작용
    # 치료를 받는 사람의 행동 패턴 가중치
    df['behavior_treatment'] = df['behavior_index'] * (1 + df['treatment'])

    return df

def add_group_stats_no_leakage(train, test):
    train = train.copy()
    test = test.copy()
    bins = [60, 80, 90, 100, 130]
    labels = ["60-79", "80-89", "90-99", "100+"]
    for df in [train, test]:
        df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=False, include_lowest=True)

    group_cols = ['disease_stage', 'gender', 'age_group']
    agg_cols = ['enzyme_A', 'enzyme_B', 'protein_level', 'immune_index', 'lipid_index', 'blood_cells']

    for group_col in group_cols:
        for col in agg_cols:
            train_groupby = train.groupby(group_col)[col]
            mean_map = train_groupby.mean()
            std_map  = train_groupby.std()
            prefix = f'{col}_by_{group_col}'
            for df_target in [train, test]:
                mapped_mean = df_target[group_col].map(mean_map).astype(float).values
                mapped_std  = df_target[group_col].map(std_map).astype(float).values
                col_values  = df_target[col].astype(float).values
                df_target[f'{prefix}_diff_mean']  = col_values - mapped_mean
                df_target[f'{prefix}_zscore']     = (col_values - mapped_mean) / (mapped_std + 1e-6)
                df_target[f'{prefix}_ratio_mean'] = col_values / (mapped_mean + 1e-6)
    return train, test

def add_qbins_no_leakage(train, test, n_bins=4):
    train = train.copy()
    test = test.copy()
    lab_cols = ["chem_01", "chem_02", "protein_level", "trace_metal", "enzyme_A", "enzyme_B", "lipid_index", "blood_cells", "clot_time"]
    for col in lab_cols:
        if col not in train.columns: continue
        try:
            _, bins = pd.qcut(train[col], q=n_bins, retbins=True, duplicates='drop')
        except ValueError:
            continue
        train[f"{col}_qbin"] = pd.cut(train[col], bins=bins, labels=False, include_lowest=True).fillna(-1).astype(int)
        test[f"{col}_qbin"] = pd.cut(test[col], bins=bins, labels=False, include_lowest=True).fillna(-1).astype(int)
    return train, test

# ----------------------------------------------------
# 3. 전처리 실행
# ----------------------------------------------------
print("Preprocessing Start...")
train_df = prepare_raw_simple(train_df)
test_df = prepare_raw_simple(test_df)

train_df = add_eda_rule_features_v2(train_df)
test_df  = add_eda_rule_features_v2(test_df)

train_df = make_boosting_features_v2(train_df)
test_df  = make_boosting_features_v2(test_df)

train_df = add_discrete_caution_features(train_df)
test_df  = add_discrete_caution_features(test_df)

train_df = add_post_eda_features(train_df)
test_df  = add_post_eda_features(test_df)

train_df = add_missing_health_features(train_df)
test_df  = add_missing_health_features(test_df)

train_df = add_ml_health_features(train_df)
test_df  = add_ml_health_features(test_df)

train_df, test_df = add_group_stats_no_leakage(train_df, test_df)
train_df, test_df = add_qbins_no_leakage(train_df, test_df)

drop_cols = ["index", "name", "birth_date", "geo_code", "birth_date"]
y = train_df["target"].values
X_all = train_df.drop(columns=drop_cols + ["target"], errors='ignore')
X_all_test = test_df.drop(columns=drop_cols, errors='ignore')

print("Preprocessing Done.")

# ----------------------------------------------------
# 4. 스마트 스케일링 (Skewness 기반)
# ----------------------------------------------------
numeric_candidates = X_all.select_dtypes(include=['number']).columns
EXCLUDE_COLS = [
    "gender_int", "fluid_accum", "organ_enlarge", "vascular_marks", 
    "swelling", "treatment", "geo_code", "risk_segment",
    "chem_01_range", "trace_metal_range", "swelling_ord",
    "target", "index"
]
NUMERIC_COLS = [c for c in numeric_candidates if c not in EXCLUDE_COLS and not c.endswith('_qbin')]

def apply_skew_scaling(train_df, test_df, cols):
    train_df = train_df.copy()
    test_df = test_df.copy()
    skewness = train_df[cols].skew()
    print(f"Applying Smart Scaling on {len(cols)} columns...")
    
    for col in cols:
        skew = abs(skewness[col])
        if skew < 0.5:
            scaler = StandardScaler()
        elif skew < 1.5:
            scaler = RobustScaler()
        else:
            scaler = QuantileTransformer(output_distribution="normal", random_state=RANDOM_STATE)
        train_df[col] = scaler.fit_transform(train_df[[col]]).flatten()
        test_df[col] = scaler.transform(test_df[[col]]).flatten()
    return train_df, test_df

X_all, X_all_test = apply_skew_scaling(X_all, X_all_test, NUMERIC_COLS)

Train shape: (6500, 24)
Test shape:  (1405, 23)
Preprocessing Start...
Preprocessing Done.
Applying Smart Scaling on 105 columns...


In [9]:
# ----------------------------------------------------
# 5. 최적 피처 및 하이퍼파라미터 설정
# ----------------------------------------------------

# [XGBoost]
XGB_FEATS_SELECTED = [
    'chem_01_range', 'chem01_trace_combo', 'chem_01', 'symptom_count', 'clot_time_qbin', 
    'symptom_severity', 'health_per_stage', 'obs_days', 'swelling_ord', 'gender_int', 
    'age', 'lipid_index_by_gender_ratio_mean', 'chem_01_qbin', 'enzyme_B_by_disease_stage_zscore', 
    'blood_cells_by_gender_zscore', 'enzyme_A_by_gender_diff_mean', 'chem01_treatment_interaction', 
    'deadly_combo', 'enzyme_A_by_disease_stage_ratio_mean', 'enzyme_B', 'sum_enzyme', 
    'chem01_chem02_combo', 'sum_chem', 'blood_cells_by_gender_ratio_mean', 
    'trace_treatment_interaction', 'enzyme_B_by_age_group_ratio_mean', 
    'lipid_index_by_disease_stage_ratio_mean', 'blood_cells_by_age_group_diff_mean', 
    'enzyme_A', 'enzyme_B_by_disease_stage_diff_mean', 'blood_treatment_interaction', 
    'blood_cells_by_disease_stage_diff_mean', 'enzyme_ratio', 'behavior_per_age', 
    'lipid_index_qbin', 'treatment'
]

final_params_xgb = {
    'n_estimators': 788, 'learning_rate': 0.06238066751566709, 'max_depth': 6,
    'min_child_weight': 1, 'gamma': 0.3960858015669083, 'subsample': 0.9214205395769955,
    'colsample_bytree': 0.5491320112459366, 'reg_alpha': 0.002669920381142417,
    'reg_lambda': 0.015444177908340267,
    'objective': 'multi:softprob', 'eval_metric': 'mlogloss',
    'tree_method': 'hist', 'random_state': RANDOM_STATE, 'n_jobs': -1
}

# [LightGBM]
LGBM_FEATS_SELECTED = [
    'obs_days', 'clot_time', 'age', 'chem02_trace_ratio', 'lipid_blood_ratio', 
    'disease_velocity', 'trace_metal', 'chem01_treatment_interaction', 
    'blood_treatment_interaction', 'health_per_stage', 'trace_treatment_interaction', 
    'chem_02', 'diff_chem', 'diff_enzyme', 'enzyme_B_by_age_group_diff_mean', 
    'sum_enzyme', 'enzyme_A_by_gender_zscore', 'log_obs_days', 'obs_days_squared'
]

final_params_lgb = {
    'n_estimators': 912, 'learning_rate': 0.01090933368427726, 'num_leaves': 193,
    'max_depth': 14, 'min_child_samples': 51, 'subsample': 0.5933220031847126,
    'colsample_bytree': 0.5036928973563994, 'reg_alpha': 0.008835424632453443,
    'reg_lambda': 0.07034822683895961,
    'objective': 'multiclass', 'random_state': RANDOM_STATE, 'n_jobs': -1, 'verbosity': -1
}

In [13]:
# %% [검증] 선정된 피처 리스트 내에서 재최적화 (Forward Selection)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import pandas as pd
import numpy as np
NEW_ML_FEATS = [
    'immune_per_behavior', 'clot_per_cell', 'lipid_per_protein', 
    'trace_density', 'behavior_treatment'
]

# 1. 검증할 피처 리스트 정의 (사용자님이 뽑아둔 것)
XGB_FEATS_CANDIDATES = [
    'chem_01_range', 'chem01_trace_combo', 'chem_01', 'symptom_count', 'clot_time_qbin', 
    'symptom_severity', 'health_per_stage', 'obs_days', 'swelling_ord', 'gender_int', 
    'age', 'lipid_index_by_gender_ratio_mean', 'chem_01_qbin', 'enzyme_B_by_disease_stage_zscore', 
    'blood_cells_by_gender_zscore', 'enzyme_A_by_gender_diff_mean', 'chem01_treatment_interaction', 
    'deadly_combo', 'enzyme_A_by_disease_stage_ratio_mean', 'enzyme_B', 'sum_enzyme', 
    'chem01_chem02_combo', 'sum_chem', 'blood_cells_by_gender_ratio_mean', 
    'trace_treatment_interaction', 'enzyme_B_by_age_group_ratio_mean', 
    'lipid_index_by_disease_stage_ratio_mean', 'blood_cells_by_age_group_diff_mean', 
    'enzyme_A', 'enzyme_B_by_disease_stage_diff_mean', 'blood_treatment_interaction', 
    'blood_cells_by_disease_stage_diff_mean', 'enzyme_ratio', 'behavior_per_age', 
    'lipid_index_qbin', 'treatment'
] + NEW_ML_FEATS

LGBM_FEATS_CANDIDATES = [
    'obs_days', 'clot_time', 'age', 'chem02_trace_ratio', 'lipid_blood_ratio', 
    'disease_velocity', 'trace_metal', 'chem01_treatment_interaction', 
    'blood_treatment_interaction', 'health_per_stage', 'trace_treatment_interaction', 
    'chem_02', 'diff_chem', 'diff_enzyme', 'enzyme_B_by_age_group_diff_mean', 
    'sum_enzyme', 'enzyme_A_by_gender_zscore', 'log_obs_days', 'obs_days_squared'
]+ NEW_ML_FEATS

# 2. 이미 찾은 최적 파라미터 (이걸로 검증해야 정확함)
final_params_xgb = {
    'n_estimators': 788, 'learning_rate': 0.06238, 'max_depth': 6, 'min_child_weight': 1,
    'gamma': 0.3961, 'subsample': 0.9214, 'colsample_bytree': 0.5491, 'reg_alpha': 0.0027,
    'reg_lambda': 0.0154, 'objective': 'multi:softprob', 'eval_metric': 'mlogloss',
    'tree_method': 'hist', 'random_state': 42, 'n_jobs': -1, 'verbosity': 0
}

final_params_lgb = {
    'n_estimators': 912, 'learning_rate': 0.0109, 'num_leaves': 193, 'max_depth': 14,
    'min_child_samples': 51, 'subsample': 0.5933, 'colsample_bytree': 0.5037,
    'reg_alpha': 0.0088, 'reg_lambda': 0.0703, 'objective': 'multiclass',
    'random_state': 42, 'n_jobs': -1, 'verbosity': -1
}

# 3. Forward Selection 함수
def run_forward_verification(model_name, model, candidate_feats, X, y):
    print(f"\n🚀 [{model_name}] 재검증 시작 ({len(candidate_feats)}개 후보)...")
    
    # 후보 피처가 실제 데이터에 있는지 확인
    valid_candidates = [f for f in candidate_feats if f in X.columns]
    
    best_score = 0.0
    best_subset = []
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # 순서는 기존 리스트 순서대로(중요도 순) 진행
    for i, feature in enumerate(valid_candidates):
        current_features = best_subset + [feature]
        X_subset = X[current_features].values
        
        fold_scores = []
        for train_idx, val_idx in kf.split(X_subset, y):
            model.fit(X_subset[train_idx], y[train_idx])
            preds = model.predict_proba(X_subset[val_idx])
            
            if preds.shape[1] > 2:
                y_bin = label_binarize(y[val_idx], classes=np.unique(y))
                score = average_precision_score(y_bin, preds, average="macro")
            else:
                score = average_precision_score(y[val_idx], preds[:, 1], average="macro")
            fold_scores.append(score)
            
        mean_score = np.mean(fold_scores)
        
        if mean_score > best_score:
            diff = mean_score - best_score
            best_score = mean_score
            best_subset.append(feature)
            print(f"[{i+1}/{len(valid_candidates)}] + {feature:<30} | Score: {mean_score:.5f} (▲ {diff:.5f})")
        else:
            # 점수 안 오르면 과감히 제외 (Pruning)
            print(f"[{i+1}/{len(valid_candidates)}] - {feature:<30} | Score: {mean_score:.5f} (Pass)")
            pass
            
    print(f"\n✅ [{model_name}] 최종 결과: {len(best_subset)}개 선택됨 (기존 {len(valid_candidates)}개)")
    print(f"🏆 Best mAP: {best_score:.6f}")
    print(f"Selected Features: {best_subset}")
    return best_subset

# 4. 실행
# (X_all, y는 전처리 완료된 상태여야 함)
xgb_model = XGBClassifier(**final_params_xgb)
xgb_final_feats = run_forward_verification("XGBoost", xgb_model, XGB_FEATS_CANDIDATES, X_all, y)

lgb_model = LGBMClassifier(**final_params_lgb)
lgb_final_feats = run_forward_verification("LightGBM", lgb_model, LGBM_FEATS_CANDIDATES, X_all, y)


🚀 [XGBoost] 재검증 시작 (41개 후보)...
[1/41] + chem_01_range                  | Score: 0.47901 (▲ 0.47901)
[2/41] + chem01_trace_combo             | Score: 0.55180 (▲ 0.07280)
[3/41] + chem_01                        | Score: 0.56177 (▲ 0.00997)
[4/41] + symptom_count                  | Score: 0.59189 (▲ 0.03012)
[5/41] + clot_time_qbin                 | Score: 0.60616 (▲ 0.01427)
[6/41] + symptom_severity               | Score: 0.61015 (▲ 0.00399)
[7/41] - health_per_stage               | Score: 0.60850 (Pass)
[8/41] + obs_days                       | Score: 0.64084 (▲ 0.03069)
[9/41] - swelling_ord                   | Score: 0.64081 (Pass)
[10/41] + gender_int                     | Score: 0.64193 (▲ 0.00109)
[11/41] + age                            | Score: 0.65896 (▲ 0.01703)
[12/41] + lipid_index_by_gender_ratio_mean | Score: 0.66159 (▲ 0.00262)
[13/41] - chem_01_qbin                   | Score: 0.65926 (Pass)
[14/41] + enzyme_B_by_disease_stage_zscore | Score: 0.66651 (▲ 0.00492)
[15/41] 

KeyboardInterrupt: 

In [ ]:
# 데이터셋 준비
X_xgb = X_all[XGB_FEATS_SELECTED].copy()
X_xgb_test = X_all_test[XGB_FEATS_SELECTED].copy()

X_lgb = X_all[LGBM_FEATS_SELECTED].copy()
X_lgb_test = X_all_test[LGBM_FEATS_SELECTED].copy()

# ----------------------------------------------------
# 6. OOF 생성 및 모델 학습
# ----------------------------------------------------
def get_oof_predictions(model, X, y, X_test, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    oof_train = np.zeros((X.shape[0], 3))
    oof_test = np.zeros((X_test.shape[0], 3))
    
    # .values 확인
    X_val = X.values if hasattr(X, 'values') else X
    X_te_val = X_test.values if hasattr(X_test, 'values') else X_test
    
    for train_idx, val_idx in skf.split(X_val, y):
        X_tr, y_tr = X_val[train_idx], y[train_idx]
        X_v = X_val[val_idx]
        
        clf = clone(model)
        clf.fit(X_tr, y_tr)
        
        oof_train[val_idx] = clf.predict_proba(X_v)
        oof_test += clf.predict_proba(X_te_val) / n_splits
    return oof_train, oof_test

print("🚀 OOF Training Start...")
xgb_model = XGBClassifier(**final_params_xgb)
lgb_model = LGBMClassifier(**final_params_lgb)

oof_train_xgb, oof_test_xgb = get_oof_predictions(xgb_model, X_xgb, y, X_xgb_test, n_splits=N_SPLITS)
oof_train_lgb, oof_test_lgb = get_oof_predictions(lgb_model, X_lgb, y, X_lgb_test, n_splits=N_SPLITS)
print("OOF Done.")




Train shape: (6500, 24)
Test shape:  (1405, 23)
Preprocessing Start...
Preprocessing Done.
Applying Smart Scaling on 99 columns...
🚀 OOF Training Start...


KeyboardInterrupt: 

In [ ]:

# ----------------------------------------------------
# 7. 메타 모델 (Stacking) & 가중치 앙상블 (Weighted)
# ----------------------------------------------------
X_meta_train = np.hstack([oof_train_xgb, oof_train_lgb])
X_meta_test = np.hstack([oof_test_xgb, oof_test_lgb])

# (1) Stacking (Logistic Regression)
# Meta Params from tuning: C=0.58855...
meta_model = LogisticRegression(C=0.5885532096799948, solver='lbfgs', random_state=RANDOM_STATE, max_iter=1000)
meta_model.fit(X_meta_train, y)
pred_stacking = meta_model.predict_proba(X_meta_test)

# (2) Weighted Blending
# Weights from tuning: XGB(0.59) / LGB(0.41) (based on your previous logs/assumption)
# 또는 랭크 앙상블(0.75 / 0.25) 등
W_XGB = 0.60
W_LGB = 0.40
pred_weighted = (oof_test_xgb * W_XGB) + (oof_test_lgb * W_LGB)

# (3) Rank Ensemble
def get_rank(proba):
    ranks = np.zeros_like(proba)
    for i in range(proba.shape[1]):
        ranks[:, i] = rankdata(proba[:, i]) / len(proba)
    return ranks

# Rank Weights (Example: XGB 0.75, LGB 0.25 from log)
R_XGB = 0.75
R_LGB = 0.25
rank_xgb = get_rank(oof_test_xgb)
rank_lgb = get_rank(oof_test_lgb)
pred_rank = (rank_xgb * R_XGB) + (rank_lgb * R_LGB)


In [4]:
# ====================================================
# [수정] 최종 평가 및 MAP 점수 출력 (완전판)
# ====================================================

from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
from scipy.stats import rankdata
import numpy as np

# 1. mAP 평가 함수 정의 (누락된 부분 추가)
def score_function(y_true, y_pred_proba):
    # 클래스가 3개 이상인 경우 원-핫 인코딩 후 mAP 계산
    if y_pred_proba.shape[1] > 2:
        y_bin = label_binarize(y_true, classes=np.unique(y_true))
        return average_precision_score(y_bin, y_pred_proba, average="macro")
    else:
        return average_precision_score(y_true, y_pred_proba[:, 1], average="macro")

# ----------------------------------------------------
# 7. 앙상블 및 성능 평가 (CV MAP 계산)
# ----------------------------------------------------
print("\n📊 앙상블 모델별 CV MAP 점수 평가 중...\n")

# 메타 데이터 (XGB OOF + LGB OOF)
if 'oof_train_xgb' in locals() and 'oof_train_lgb' in locals():
    X_meta_train = np.hstack([oof_train_xgb, oof_train_lgb])

    # (1) Stacking Ensemble 평가
    meta_model = LogisticRegression(
        C=0.5885532096799948, 
        solver='lbfgs', 
        random_state=42, 
        max_iter=1000
    )

    meta_oof_pred = cross_val_predict(
        meta_model, 
        X_meta_train, 
        y, 
        cv=5, 
        method='predict_proba', 
        n_jobs=-1
    )
    score_stacking = score_function(y, meta_oof_pred)
    print(f"🏆 [Stacking Ensemble] CV MAP: {score_stacking:.6f}")


    # (2) Weighted Ensemble 평가
    W_XGB = 0.60
    W_LGB = 0.40

    # Train OOF에 가중치 적용
    weighted_oof_pred = (oof_train_xgb * W_XGB) + (oof_train_lgb * W_LGB)
    score_weighted = score_function(y, weighted_oof_pred)
    print(f"⚖️ [Weighted Ensemble] CV MAP: {score_weighted:.6f}")


    # (3) Rank Ensemble 평가
    R_XGB = 0.75
    R_LGB = 0.25

    def get_rank(proba):
        ranks = np.zeros_like(proba)
        for i in range(proba.shape[1]):
            # 각 클래스별 확률을 순위(0~1)로 변환
            ranks[:, i] = rankdata(proba[:, i]) / len(proba)
        return ranks

    rank_train_xgb = get_rank(oof_train_xgb)
    rank_train_lgb = get_rank(oof_train_lgb)

    rank_oof_pred = (rank_train_xgb * R_XGB) + (rank_train_lgb * R_LGB)
    score_rank = score_function(y, rank_oof_pred)
    print(f"🥇 [Rank Ensemble]     CV MAP: {score_rank:.6f}")

    print("\n✅ 평가 완료!")




📊 앙상블 모델별 CV MAP 점수 평가 중...

🏆 [Stacking Ensemble] CV MAP: 0.690088
⚖️ [Weighted Ensemble] CV MAP: 0.700440
🥇 [Rank Ensemble]     CV MAP: 0.698298

✅ 평가 완료!
